In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from math import sin, cos, sqrt, atan2, radians
import re
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


In [3]:
calgary_df = pd.read_csv('data/calgary_listings.csv')
calgary_df.head()

,price,postal_code,address,mls,bed,full_bath,half_bath,property_area,property_type,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments,lot_size,basement,garage
0,597900,T3P0K3,100 EVANSFIELD PL NW,A1046587,4,3,1.0,2467,house,51.177351,-114.110308,2012.0,8,14.94,2585.39,5468,1,1
1,319900,T2W1G1,10304 ELBOW DR SW,A1046733,3,1,NaN,1040,house,50.961657,-114.084377,1961.0,59,9.29,1383.29,5500,1,1
2,349000,T3M1C3,110 CRAMOND PL SE,A1046724,3,2,1.0,1245,house,50.893400,-113.979031,2001.0,19,18.12,1509.12,2669,1,1
3,329900,T3L3C2,15 Toscana Gardens NW,A1046451,2,2,1.0,1345,townhouse,51.121067,-114.254993,2004.0,16,15.22,1426.53,na,0,1
4,334900,T3G4J5,155 Citadel Drive NW,A1044809,3,2,1.0,1389,townhouse,51.142426,-114.180925,1995.0,25,13.17,1448.15,2314,1,1


In [4]:
post_code_list = []

for i in calgary_df["postal_code"]:
    post_code_list.append(i)

In [5]:
scores_walk = []
scores_bike = []
scores_transit = []

for i in post_code_list:

    time.sleep(5)
    
    try:
        postal_code = i.replace(" ", "%20")
        url_score = "https://www.walkscore.com/score/" + str(postal_code)
        time.sleep(5)

        # Parse HTML with Beautiful Soup
        response = requests.get(url_score)
        code_soup = BeautifulSoup(response.text, 'html.parser')

        if 'pp.walk.sc/badge/walk/score' in str(code_soup):
            ws = str(code_soup).split('pp.walk.sc/badge/walk/score/')[1][:2].replace('.','')
            scores_walk.append(ws)
        else:
            ws = 'N/A'
            scores_walk.append(ws)
        if 'pp.walk.sc/badge/bike/score' in str(code_soup):
            bs = str(code_soup).split('pp.walk.sc/badge/bike/score/')[1][:2].replace('.','')
            scores_bike.append(bs)
        else:
            bs = 'N/A'
            scores_bike.append(bs)
        if 'pp.walk.sc/badge/transit/score' in str(code_soup):
            ts = str(code_soup).split('pp.walk.sc/badge/transit/score/')[1][:2].replace('.','')
            scores_transit.append(ts)
        else:
            ts = 'N/A'
            scores_transit.append(ts)
    except:
        ws = 'N/A'
        scores_walk.append(ws)
        bs = 'N/A'
        scores_bike.append(bs)
        ts = 'N/A'
        scores_transit.append(ts)

In [6]:
score_df_trans = {'postal_code':post_code_list, 
                  'walk_score':scores_walk, 
                  'bike_score':scores_bike, 
                  'transit_score':scores_transit}
score_df_dup = pd.DataFrame(score_df_trans)
score_df = score_df_dup.drop_duplicates()
score_df.head()

,postal_code,walk_score,bike_score,transit_score
0,T3P0K3,N/A,N/A,N/A
1,T2W1G1,64,76,65
2,T3M1C3,23,83,33
3,T3L3C2,3,61,28
4,T3G4J5,41,91,33


In [7]:
score_df.to_csv('data/score_df.csv', index=False)